In [1]:
# Josue Castellanos (2024)
# This script is for interferogram analysis as part of the PX Beamlines LDRD Project
import os
from glob import glob
import os
from PIL import Image
import pandas as pd
from io import StringIO
import numpy as np


# carpenmentalize the process to work with a certain data. 

# put images together as a motor coordinate, could be negative

# 3 or 4th data file in txt have coordinates

# Get coordinates from txt, associate the coordinates to KCube DC 2 Goal, Actual (Actual is the best alignment)

# Rotate - left side is west , right side is east, 90 degree rotation -90 or +90

# X motor is negative

# what direction is it scanning?

# radius coparison 

# Stitch - actual distance and camera, effective pixel size , subtract one image from another to see the difference

# Where are we on the surface, scan 601 , lets pass into the code the cropped rotated image that has the coordinate on the file name so we can reference that to where it is in space


In [2]:
def GlobFileNames(dirNum):
    # basepath = '/Volumes/SanDisk/Image Single Motor Scan'  # My sandisk drive
    basepath = '/Beamline Controls/BCS Setup Data/240625/Image Single Motor Scan'  #PSDEV
    
    dir_path = f'{basepath} 000{dirNum} Images/*.Png'
    
    # if not os.path.isfile(dir_path):
    #     print(f"File not found: {dir_path}")
    #     return
    
    file_list = glob(dir_path)  # Sort list
    file_list.sort()
    
    return file_list

In [3]:
def ExtractData(fp):
    # Read the file content
    with open(fp, 'r') as file:
        lines = file.readlines()

    # Find the line where DATA starts
    for i, line in enumerate(lines):
        if 'DATA' in line:
            data_start_idx = i + 1
            break
    
    return data_start_idx, lines

In [4]:
def ReadData2DF(start_index, lines):
    # Read the data into a pandas DataFrame
    data_lines = lines[start_index:]
    data_str = ''.join(data_lines)
    
    # Convert the data into a DataFrame
    data = StringIO(data_str)
    df = pd.read_csv(data, sep='\t')

    return df

In [5]:
def CleanData(dict, column):
    df = dict.iloc[:,column - 1].tolist()
    
    return df

In [6]:
def CropRotateImagesToRegion(file, startXpixel, endXpixel, startYpixel, endYpixel, coordinate, save_directory):
    def _ModifyFilename(file):
        basename = os.path.basename(file)
        # Remove a specific part of the string
        modified_basename = basename.replace('Image Single Motor Scan ', '')
        modified_basename = modified_basename.replace(' ', '_')

        return modified_basename
    # Open an image file
    with Image.open(file) as img:
        # Crop the image
        cropped_img = img.crop((startXpixel, startYpixel, endXpixel, endYpixel))
        # Rotate the image 90 degrees, Pillow 90 default is counterclockwise so -90 is needed for clockwise
        rotated_img = cropped_img.rotate(-90, expand=True)
        # Modify the new file name of image
        mod_name = _ModifyFilename(file)
        # Save the cropped image
        cropped_file = os.path.join(save_directory, f"Cropped_Rotated_{coordinate}mm_" + mod_name)
        cropped_file = cropped_file.replace("\\", "/")
        rotated_img.save(cropped_file)
        print(f"Cropped image saved as: {cropped_file}")
        
        return cropped_file

In [7]:
def CropImages(files, startXpixel, endXpixel, startYpixel, endYpixel, coords, save_directory):
    cropped_files = []
    for file, coordinate in zip(files, coords):
        cropped_file = CropRotateImagesToRegion(file, startXpixel, endXpixel, startYpixel, endYpixel, coordinate, save_directory)
        if cropped_file:
            cropped_files.append(cropped_file)

    return cropped_files

In [8]:
def Subtract2Images(image1_path, image2_path, index1, index2, output_path):
    # Open the first image
    with Image.open(image1_path) as img1:
        img1_array = np.array(img1)
    
    # Open the second image
    with Image.open(image2_path) as img2:
        img2_array = np.array(img2)
    
    # Check both images have the same dimensions
    if img1_array.shape != img2_array.shape:
        raise ValueError("The images must have the same dimensions")

    # Subtract the two images
    subtracted_array = img1_array - img2_array
    
    # Clip the values to be in the range [0, 255]
    subtracted_array = np.clip(subtracted_array, 0, 255)
    
    # Convert the result back to a PIL image
    subtracted_image = Image.fromarray(subtracted_array.astype(np.uint8))
    
    # Save the resulting image to direcotry
    sub_file = os.path.join(output_path, f"Subtracted_{index2}_From_{index1}.Png")

    subtracted_image.save(sub_file, format='PNG')
    print(f'Subtracted image saved as: {output_path}')


In [9]:
def ReadModifiedImages(dirNum):
    # Read the cropped images from disk and return them as a list
    cropped_files = [os.path.join(dirNum, f) for f in os.listdir(dirNum) if "Cropped_" in f]

    return cropped_files

In [10]:
# Below is testing the functionality
# Replace with the desired directory number 000599, 000600, 000601...etc
file_index = 601
file_names = GlobFileNames(file_index)

# Check if files are correct
for f in file_names:
    print(f)

/Beamline Controls/BCS Setup Data/240625/Image Single Motor Scan 000601 Images\Image Single Motor Scan 000601 2D Image 001.Png
/Beamline Controls/BCS Setup Data/240625/Image Single Motor Scan 000601 Images\Image Single Motor Scan 000601 2D Image 002.Png
/Beamline Controls/BCS Setup Data/240625/Image Single Motor Scan 000601 Images\Image Single Motor Scan 000601 2D Image 003.Png
/Beamline Controls/BCS Setup Data/240625/Image Single Motor Scan 000601 Images\Image Single Motor Scan 000601 2D Image 004.Png
/Beamline Controls/BCS Setup Data/240625/Image Single Motor Scan 000601 Images\Image Single Motor Scan 000601 2D Image 005.Png
/Beamline Controls/BCS Setup Data/240625/Image Single Motor Scan 000601 Images\Image Single Motor Scan 000601 2D Image 006.Png
/Beamline Controls/BCS Setup Data/240625/Image Single Motor Scan 000601 Images\Image Single Motor Scan 000601 2D Image 007.Png
/Beamline Controls/BCS Setup Data/240625/Image Single Motor Scan 000601 Images\Image Single Motor Scan 000601 2

In [11]:
# Replace with the actual path to the text file
header_path = (f'/Beamline Controls/BCS Setup Data/240625/Image Single Motor Scan 000{file_index}.txt' ) 

# Extract All lines in txt file and provide a start index 
start_index, lines = ExtractData(header_path)
data = ReadData2DF(start_index, lines)

column = 4    # Column in dataframe
coordinates = CleanData(data, column)
coordinates

[49.70000076,
 49.59999847,
 49.5,
 49.40000153,
 49.29999924,
 49.20000076,
 49.09999847,
 49.0,
 48.90000153,
 48.79999924,
 48.70000076,
 48.59999847,
 48.5,
 48.40000153,
 48.29999924,
 48.20000076,
 48.09999847,
 48.0,
 47.90000153,
 47.79999924,
 47.70000076,
 47.59999847,
 47.5,
 47.40000153,
 47.29999924,
 47.20000076,
 47.09999847,
 47.0,
 46.90000153,
 46.79999924,
 46.70000076,
 46.59999847,
 46.5,
 46.40000153,
 46.29999924,
 46.20000076,
 46.09999847,
 46.0,
 45.90000153,
 45.79999924,
 45.70000076,
 45.59999847,
 45.5,
 45.40000153,
 45.29999924,
 45.20000076,
 45.09999847,
 45.0,
 44.90000153,
 44.79999924,
 44.70000076]

In [12]:
startXpixel = 780
endXpixel = 2500
startYpixel = 10
endYpixel = 1400
date = 240625
save_directory = f'/Beamline Controls/BCS Setup Data/{date}/Image Single Motor Scan 000{file_index} Images'

cropped_files = CropImages(file_names, startXpixel, endXpixel, startYpixel, endYpixel, coordinates, save_directory)


Cropped image saved as: /Beamline Controls/BCS Setup Data/240625/Image Single Motor Scan 000601 Images/Cropped_Rotated_49.70000076mm_000601_2D_Image_001.Png
Cropped image saved as: /Beamline Controls/BCS Setup Data/240625/Image Single Motor Scan 000601 Images/Cropped_Rotated_49.59999847mm_000601_2D_Image_002.Png
Cropped image saved as: /Beamline Controls/BCS Setup Data/240625/Image Single Motor Scan 000601 Images/Cropped_Rotated_49.5mm_000601_2D_Image_003.Png
Cropped image saved as: /Beamline Controls/BCS Setup Data/240625/Image Single Motor Scan 000601 Images/Cropped_Rotated_49.40000153mm_000601_2D_Image_004.Png
Cropped image saved as: /Beamline Controls/BCS Setup Data/240625/Image Single Motor Scan 000601 Images/Cropped_Rotated_49.29999924mm_000601_2D_Image_005.Png
Cropped image saved as: /Beamline Controls/BCS Setup Data/240625/Image Single Motor Scan 000601 Images/Cropped_Rotated_49.20000076mm_000601_2D_Image_006.Png
Cropped image saved as: /Beamline Controls/BCS Setup Data/240625/

In [13]:
img1 = 599
img2 = 600
img1_path = f'/Beamline Controls/BCS Setup Data/{date}/Image Single Motor Scan 000{img1} Images'
img2_path = f'/Beamline Controls/BCS Setup Data/{date}/Image Single Motor Scan 000{img2} Images'
save_path = f'/Beamline Controls/BCS Setup Data/{date}/Subtracted Images'

images1 = ReadModifiedImages(img1_path)
images2 = ReadModifiedImages(img2_path)

Subtract2Images(images1[0], images2[0], img2, img1, save_path)

Subtracted image saved as: /Beamline Controls/BCS Setup Data/240625/Subtracted Images


**TESTING BELOW**